In [1]:
import os
import json
import pandas as pd
import traceback

In [2]:
from langchain_ai21 import ChatAI21
from dotenv import load_dotenv
load_dotenv()


True

In [3]:
Key = os.getenv("AI21_API_KEY")

In [4]:
print("Key: ", Key)

Key:  jN8sn36jtKAbayRcW6EitKyVz3iZySui


In [5]:
llm = ChatAI21(api_key= Key, model = "jamba-instruct-preview",temperature=0.5,max_tokens=1024)

In [6]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback

In [7]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer"
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer"
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer"
    }

}

In [8]:
TEMPLATE = """
Text:{text}
You are an expert MCQ maker.Given the above text, it is your job to \
create a quiz of {number} multiple choice questions for {subject} students in {tone} tone.
Make sure the questions are not repeated and check all the questions to be conforming the text as well. 
Make sure to format your response like RESPONSE_JSON below and use it as a guide. \
Ensure to make {number} MCQs.
### RESPONSE_JSON
{response_json}
"""

In [9]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text","number","subject","tone","response_json"],
    template=TEMPLATE
)

In [10]:
quiz_chain = LLMChain(llm = llm,prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

d:\GenAI Course\env\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [11]:
TEMPLATE2 = """
you are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students. \
You need to evaluate the complexity of the question and give a completer analysis of the quiz. Only use at max 50 words for complexity.
if the quiz is not at par with the cognitive and analytical abilities of the students, \
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the students' level.
Quiz_MCQ:
{quiz}

Check from an expert English writer of the above quiz:
"""

In [12]:
quiz_evaluation_prompt = PromptTemplate(input_variables=["subject","quiz"], template=TEMPLATE2) 

In [13]:
review_chain = LLMChain(llm = llm,prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [14]:
generate_evaluate_chain = SequentialChain(chains = [quiz_chain, review_chain], input_variables=["text","number","subject","tone","response_json"],
                                          output_variables=["quiz","review"], verbose=True)

In [15]:
file_path = r"D:\GenAI Course\data.txt"
file_path

'D:\\GenAI Course\\data.txt'

In [16]:
with open(file_path, 'r') as file:
    text = file.read()

In [17]:
print(text)

The Sun is a G-type main-sequence star that makes up about 99.86% of the mass of the Solar System.[27] It has an absolute magnitude of +4.83, estimated to be brighter than about 85% of the stars in the Milky Way, most of which are red dwarfs.[28][29] It is more massive than 95% of nearby stars within 7 pc. (~23lt yrs)[30] The Sun is a Population I, or heavy-element-rich,[b] star.[31] Its formation approximately 4.6 billion years ago may have been triggered by shockwaves from one or more nearby supernovae.[32][33] This is suggested by a high abundance of heavy elements in the Solar System, such as gold and uranium, relative to the abundances of these elements in so-called Population II, heavy-element-poor, stars. The heavy elements could most plausibly have been produced by endothermic nuclear reactions during a supernova, or by transmutation through neutron absorption within a massive second-generation star.[31]

The Sun is by far the brightest object in the Earth's sky, with an appare

In [18]:
# Serialize the Python dictionary into a JSON formatted string
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [19]:
NUMBER = 5
SUBJECT = "Sun"
TONE = "simple"

In [20]:
# with get_openai_callback() as cb:
#     response = generate_evaluate_chain(
#         {
#             "text": text,
#             "number": NUMBER,
#             "subject": SUBJECT,
#             "tone": TONE,
#             "response_json": json.dumps(RESPONSE_JSON),
#         }
#     )

In [20]:
# Generate the response using AI21 model
try:
    response = generate_evaluate_chain(
        {
            "text": text,
            "number": NUMBER,
            "subject": SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON),
        }
    )
    
    # Print the full response for debugging
    print("Full API Response:")
    print(json.dumps(response, indent=2))
    
    # Try to parse the quiz and review
    try:
        quiz = json.loads(response["quiz"])
        print("Parsed Quiz:")
        print(json.dumps(quiz, indent=2))
    except json.JSONDecodeError:
        print("Failed to parse quiz as JSON. Raw quiz output:")
        print(response["quiz"])
    
    try:
        review = json.loads(response["review"])
        print("Parsed Review:")
        print(json.dumps(review, indent=2))
    except json.JSONDecodeError:
        print("Failed to parse review as JSON. Raw review output:")
        print(response["review"])

except Exception as e:
    print(f"An error occurred: {str(e)}")
    print(traceback.format_exc())

d:\GenAI Course\env\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:The Sun is a G-type main-sequence star that makes up about 99.86% of the mass of the Solar System.[27] It has an absolute magnitude of +4.83, estimated to be brighter than about 85% of the stars in the Milky Way, most of which are red dwarfs.[28][29] It is more massive than 95% of nearby stars within 7 pc. (~23lt yrs)[30] The Sun is a Population I, or heavy-element-rich,[b] star.[31] Its formation approximately 4.6 billion years ago may have been triggered by shockwaves from one or more nearby supernovae.[32][33] This is suggested by a high abundance of heavy elements in the Solar System, such as gold and uranium, relative to the abundances of these elements in so-called Population II, heavy-element-poor, stars. The heavy elements could most plausibly have been produced by endothermic nuclear reactions during a supernova, or by transmutation through neutron absorption within a ma

HTTP Request: POST https://api.ai21.com/studio/v1/chat/completions "HTTP/1.1 200 OK"



> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:

you are an expert english grammarian and writer. Given a Multiple Choice Quiz for Sun students. You need to evaluate the complexity of the question and give a completer analysis of the quiz. Only use at max 50 words for complexity.
if the quiz is not at par with the cognitive and analytical abilities of the students, update the quiz questions which needs to be changed and change the tone such that it perfectly fits the students' level.
Quiz_MCQ:
{"1": {"mcq": "What type of star is the Sun?", "options": {"a": "A-type main-sequence star", "b": "G-type main-sequence star", "c": "K-type main-sequence star", "d": "F-type main-sequence star"}, "correct": "b"}}
{"2": {"mcq": "How does the Sun's rotation rate compare to its early state?", "options": {"a": "It is much faster now than in the past.", "b": "It is much slower now than in the past.", "c": "It has not changed significantly.", "d": "It is difficult to dete

HTTP Request: POST https://api.ai21.com/studio/v1/chat/completions "HTTP/1.1 200 OK"



> Finished chain.

> Finished chain.
Full API Response:
{
  "text": "The Sun is a G-type main-sequence star that makes up about 99.86% of the mass of the Solar System.[27] It has an absolute magnitude of +4.83, estimated to be brighter than about 85% of the stars in the Milky Way, most of which are red dwarfs.[28][29] It is more massive than 95% of nearby stars within 7 pc. (~23lt yrs)[30] The Sun is a Population I, or heavy-element-rich,[b] star.[31] Its formation approximately 4.6 billion years ago may have been triggered by shockwaves from one or more nearby supernovae.[32][33] This is suggested by a high abundance of heavy elements in the Solar System, such as gold and uranium, relative to the abundances of these elements in so-called Population II, heavy-element-poor, stars. The heavy elements could most plausibly have been produced by endothermic nuclear reactions during a supernova, or by transmutation through neutron absorption within a massive second-generation star.[31]\n\nT

In [21]:
response

{'text': "The Sun is a G-type main-sequence star that makes up about 99.86% of the mass of the Solar System.[27] It has an absolute magnitude of +4.83, estimated to be brighter than about 85% of the stars in the Milky Way, most of which are red dwarfs.[28][29] It is more massive than 95% of nearby stars within 7 pc. (~23lt yrs)[30] The Sun is a Population I, or heavy-element-rich,[b] star.[31] Its formation approximately 4.6 billion years ago may have been triggered by shockwaves from one or more nearby supernovae.[32][33] This is suggested by a high abundance of heavy elements in the Solar System, such as gold and uranium, relative to the abundances of these elements in so-called Population II, heavy-element-poor, stars. The heavy elements could most plausibly have been produced by endothermic nuclear reactions during a supernova, or by transmutation through neutron absorption within a massive second-generation star.[31]\n\nThe Sun is by far the brightest object in the Earth's sky, wi

In [22]:
quiz=response.get("quiz")
quiz

'{"1": {"mcq": "What type of star is the Sun?", "options": {"a": "A-type main-sequence star", "b": "G-type main-sequence star", "c": "K-type main-sequence star", "d": "F-type main-sequence star"}, "correct": "b"}}\n{"2": {"mcq": "How does the Sun\'s rotation rate compare to its early state?", "options": {"a": "It is much faster now than in the past.", "b": "It is much slower now than in the past.", "c": "It has not changed significantly.", "d": "It is difficult to determine."}, "correct": "d"}}\n{"3": {"mcq": "What is the approximate distance light travels from the Sun\'s horizon to Earth\'s horizon in minutes and seconds?", "options": {"a": "8 minutes and 20 seconds.", "b": "9 minutes and 30 seconds.", "c": "10 minutes and 10 seconds.", "d": "11 minutes and 20 seconds."}, "correct": "a"}}\n{"4": {"mcq": "What causes the differential rotation of the Sun?", "options": {"a": "Convective motion due to heat transport and the Coriolis force.", "b": "The gravitational pull of planets.", "c":

In [24]:
import json
import re

def preprocess_json(s):
    # Split the string into individual JSON objects
    json_objects = re.findall(r'\{[^{}]+\}', s)
    
    # Combine the objects into a single JSON object
    combined_json = '{' + ','.join(f'"quiz_{i+1}": {obj.strip()}' for i, obj in enumerate(json_objects)) + '}'
    
    return combined_json

# Preprocess and parse the JSON
try:
    cleaned_quiz = preprocess_json(quiz)
    quiz_data = json.loads(cleaned_quiz)
    print("JSON successfully parsed!")
    print(json.dumps(quiz_data, indent=2))  # Pretty print the parsed JSON
    
    # Save to a file
    with open('quiz.json', 'w') as f:
        json.dump(quiz_data, f, indent=2)
    print("JSON saved to 'quiz.json'")
except json.JSONDecodeError as e:
    print(f"Error parsing JSON: {e}")
    print("\nCleaned JSON string:")
    print(cleaned_quiz)
    
    print("\nAttempting to identify the problematic part:")
    lines = cleaned_quiz.split('\n')
    for i, line in enumerate(lines, 1):
        try:
            json.loads('{' + line + '}')
        except json.JSONDecodeError as e:
            print(f"Error in line {i}: {line.strip()}")
            print(f"Error message: {e}")
            break

JSON successfully parsed!
{
  "quiz_1": {
    "a": "A-type main-sequence star",
    "b": "G-type main-sequence star",
    "c": "K-type main-sequence star",
    "d": "F-type main-sequence star"
  },
  "quiz_2": {
    "a": "It is much faster now than in the past.",
    "b": "It is much slower now than in the past.",
    "c": "It has not changed significantly.",
    "d": "It is difficult to determine."
  },
  "quiz_3": {
    "a": "8 minutes and 20 seconds.",
    "b": "9 minutes and 30 seconds.",
    "c": "10 minutes and 10 seconds.",
    "d": "11 minutes and 20 seconds."
  },
  "quiz_4": {
    "a": "Convective motion due to heat transport and the Coriolis force.",
    "b": "The gravitational pull of planets.",
    "c": "The rotation of the Earth.",
    "d": "The magnetic field of the Sun."
  },
  "quiz_5": {
    "a": "\u221226.74.",
    "b": "\u22121.46.",
    "c": "5.0.",
    "d": "13 billion times brighter than Sirius."
  }
}
JSON saved to 'quiz.json'
